In [4]:
import sys
sys.path.append('../')
from transforms import DropEdges, DropEdgesWeighted
from torch_geometric.datasets import KarateClub, Planetoid
import networkx as nx
from networkx.algorithms.centrality import (
    betweenness_centrality, 
    degree_centrality
)
from torch_geometric.utils import to_networkx
import torch
import pickle as pkl

ModuleNotFoundError: No module named 'bgrl.transforms'; 'bgrl' is not a package

In [ ]:
data = KarateClub()
